In [1]:
import numbers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

%config Completer.use_jedi = False

def get_mape(y_predict, y_true):
    return (abs(y_predict - y_true) / y_true).mean()

def process_rooms_number(x):
        
    if pd.isna(x):
        return 1
    
    if isinstance(x, int):
        return x
    
    if x.isdigit():
        return int(x)
    
    if x == 'Студия':
        return 1
    
    if x == 'Своб. планировка':
        return 1
    
    if x == '> 9':
        return 10

    return 1

<h3>Реализуем линейную регрессию</h3>

<p>Чаще всего алгоритмы машинного обучения реализуются в виде классов с обязательными методами <code>.fit()</code>, <code>.predict()</code>. </p>

<p><code>.fit()</code> – обучить алгоритм на обучающей выборке;</p>

<p><code>.predict()</code> – сделать предсказание на тестовых данных.</p>

<p> </p>

In [10]:
class LinearRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100, batch_size=None, l2_coef = 0):
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        self.l2_coef = l2_coef
        
        self.weights = None
        self.bias = None
        
        self.batch_size = batch_size
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        self.check_regression_X_y(X_train, y_train)
        self.check_regression_X_y(X_val, y_val)
        
        n, m = X_train.shape
        
        self.weights = np.zeros((m, 1))
        self.bias = 0
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            
            if self.batch_size:
                random_ids = np.random.randint(0, X_train.shape[0], size=self.batch_size)
                dJdw, dJdb = self.grads(X_train[random_ids], y_train[random_ids])
            else:
                dJdw, dJdb = self.grads(X_train, y_train)
                
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = self.weights - self.lr * dJdw
            self.bias = self.bias - self.lr * dJdb
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self

    def predict(self, X):
        return X.dot(self.weights) + self.bias
    
    def grads(self, X, y):
        
        y_hat = self.predict(X)
        
        dJdw = ((y_hat - y) * X).mean(axis=0, keepdims=True).T + (self.l2_coef * self.weights)
        dJdb = (y_hat - y).mean()
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        MAPE_train = get_mape(train_preds, y_train)
        MAPE_val = get_mape(val_preds, y_val)
        
        print(f'{n_iter} completed. MAPE on train: {MAPE_train}, val: {MAPE_val},  grad norm: {gradient_norm}')
        
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным '
                             f'числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
            
        
    @staticmethod
    def check_regression_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')
                    
        if np.any([(not isinstance(value, numbers.Real)) for value in y.flatten()]):
            raise ValueError(f'Ответы на объектах должны быть действительными числами!')
       

<h3>Тестируем модель на простой задаче</h3>

In [14]:
X = np.array([
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1],
    [1, 1, 1],
])
y = np.array([[1], [2], [3], [4]])
model = LinearRegression(lr=0.1)
model.fit(X, y, X, y)
model.predict(X)

100 completed. MAPE on train: 0.5088891642967012, val: 0.5088891642967012,  grad norm: 0.0013875823192608524


array([[2.33234396],
       [2.44345507],
       [2.55456618],
       [2.66622005]])

<h3>Решаем задачу предсказания цены</h3>

In [15]:
data = pd.read_csv('real_estate_novosibirsk.csv')

<p>Чистим данные:</p>

In [16]:
data = data.drop_duplicates(subset=['item_id'], keep='last')
data = data.dropna(subset=['area'])
data['rooms_number'] = data['rooms_number'].apply(process_rooms_number).copy()
data = data[(data.price > 970000) & (data.price < 12700000)]
data = data[(data.floor < 59)]

data = data.dropna(axis=0)

In [17]:
train, val, train_price, val_price = train_test_split(data.drop('price', axis=1), data['price'], random_state=42)

In [18]:
test_data = pd.read_csv('test_price_estimator.csv')
test, test_price = test_data.drop('price', axis=1), test_data['price']

y_train = train_price.values.reshape(-1, 1)
y_val = val_price.values.reshape(-1, 1)

### Делаем бейзлайн

In [19]:
get_mape(y_predict=np.median(y_train), y_true=y_val)

0.37614684131639187

### Обучаем модель

1) Начинаем с простого

In [20]:
X_train = train[['area']].values
X_val = val[['area']].values


model = LinearRegression(lr=6e-4, max_iter=140000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.24041646500108102, val: 0.23306769336189537,  grad norm: 30726.511087846946
20000 completed. MAPE on train: 0.24589712566681887, val: 0.23837089080363735,  grad norm: 10236.612882422081
30000 completed. MAPE on train: 0.2479166434933886, val: 0.2403367969259019,  grad norm: 3410.3528059199984
40000 completed. MAPE on train: 0.24861017270627137, val: 0.24101314283237188,  grad norm: 1136.167440777104
50000 completed. MAPE on train: 0.24884343579771137, val: 0.24124089689468017,  grad norm: 378.51698253641536
60000 completed. MAPE on train: 0.24892167152761666, val: 0.2413174375000516,  grad norm: 126.10386543934003
70000 completed. MAPE on train: 0.24894775093354468, val: 0.2413429790738306,  grad norm: 42.01181350459267
80000 completed. MAPE on train: 0.24895643984622523, val: 0.24135148831207218,  grad norm: 13.996339190929458
90000 completed. MAPE on train: 0.24895933457889513, val: 0.24135432318576006,  grad norm: 4.662914890135419
100000 completed.

<p>Для того, чтобы начать ориентироваться в метрике решения задачи, очень важно построить одну или несколько простых моделей. Часто есть соблазн добавить все признаки сразу и обучить модель — мы так поступать не будем. Наоборот, мы будем постепенно добавлять признаки и следить за тем, что модель решает задачу лучше и лучше. </p>

2) Увеличиваем количество признаков

In [21]:
X_train = train[['area', 'floors_in_house', 'floor']].values
X_val = val[['area', 'floors_in_house', 'floor']].values

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train, y_train, X_val, y_val)

10000 completed. MAPE on train: 0.23871962878302797, val: 0.23047817188452926,  grad norm: 5971.194596934688
20000 completed. MAPE on train: 0.2379776705986216, val: 0.22979156736052941,  grad norm: 2981.2001659882826
30000 completed. MAPE on train: 0.2376233210814564, val: 0.22946355254736478,  grad norm: 1488.4047547623757
40000 completed. MAPE on train: 0.23745039389094072, val: 0.22930322373574852,  grad norm: 743.106329884653
50000 completed. MAPE on train: 0.23736518223078368, val: 0.22922384544850766,  grad norm: 371.0059483133193
60000 completed. MAPE on train: 0.23732283386509392, val: 0.22918441660625552,  grad norm: 185.22976880746816
70000 completed. MAPE on train: 0.23730173143630276, val: 0.2291648372041522,  grad norm: 92.47848291499318
80000 completed. MAPE on train: 0.23729120934724846, val: 0.2291550820732042,  grad norm: 46.17114115787269
90000 completed. MAPE on train: 0.23728595715888604, val: 0.2291502163088233,  grad norm: 23.05157057755291
100000 completed. MAPE

Делаем новые признаки
##### One hot encoding

In [22]:
ohe_example = pd.DataFrame({'feature': ['a', 'b', 'a', 'c']})
ohe_example

,feature
0,a
1,b
2,a
3,c


In [23]:
ohe = OneHotEncoder(sparse=False)
ohe.fit_transform(ohe_example)

array([[1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 0., 1.]])

In [24]:
ohe_house_type_transformer = OneHotEncoder(sparse=False)
train_ohe_house_type = ohe_house_type_transformer.fit_transform(train[['type_of_house']])
val_ohe_house_type = ohe_house_type_transformer.transform(val[['type_of_house']])

ohe_district_transformer = OneHotEncoder(sparse=False)
train_ohe_district = ohe_district_transformer.fit_transform(train[['district']])
val_ohe_district = ohe_district_transformer.transform(val[['district']])

X_train_extended = np.hstack([X_train, train_ohe_house_type, train_ohe_district])
X_val_extended = np.hstack([X_val, val_ohe_house_type, val_ohe_district])

model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

10000 completed. MAPE on train: 0.22941138841977596, val: 0.2214656951419709,  grad norm: 3647.0265280886033
20000 completed. MAPE on train: 0.22898330262542557, val: 0.22107901423381854,  grad norm: 1877.7821362414481
30000 completed. MAPE on train: 0.22876834702795182, val: 0.22088535115025887,  grad norm: 966.8330488208313
40000 completed. MAPE on train: 0.22865911290323174, val: 0.22078753820223476,  grad norm: 497.8033001009539
50000 completed. MAPE on train: 0.22860336310282905, val: 0.22073743719250502,  grad norm: 256.3091175811085
60000 completed. MAPE on train: 0.22857470881141922, val: 0.22071173106457354,  grad norm: 131.96851797041745
70000 completed. MAPE on train: 0.22855995928779066, val: 0.22069849753009907,  grad norm: 67.94799147105498
80000 completed. MAPE on train: 0.22855237397369385, val: 0.22069169034386688,  grad norm: 34.98508292709264
90000 completed. MAPE on train: 0.22854846956406355, val: 0.22068818826240125,  grad norm: 18.0131303501314
100000 completed. 

In [25]:
model = LinearRegression(lr=6e-4, max_iter=120000, print_every=10000, tol=0.1, batch_size=8196)
model.fit(X_train_extended, y_train, X_val_extended, y_val)

10000 completed. MAPE on train: 0.23254081234291066, val: 0.22442068758615033,  grad norm: 658037.874978649
20000 completed. MAPE on train: 0.26376928837410457, val: 0.25473641506809247,  grad norm: 22336440.81460733
30000 completed. MAPE on train: 0.22182840171636697, val: 0.2143910447481423,  grad norm: 1127192.0637011616
40000 completed. MAPE on train: 0.26831470816747927, val: 0.26663597976173176,  grad norm: 48856537.591485366
50000 completed. MAPE on train: 0.48198800924905066, val: 0.47081908071673495,  grad norm: 74294586.61583692
60000 completed. MAPE on train: 0.21644183141255355, val: 0.2094568795631596,  grad norm: 8696303.950223094
70000 completed. MAPE on train: 0.23870088069593806, val: 0.2304146701114391,  grad norm: 7402759.066630338
80000 completed. MAPE on train: 0.25759225058820023, val: 0.248726239616521,  grad norm: 14170360.493276702
90000 completed. MAPE on train: 0.22381859340688098, val: 0.21626022445365173,  grad norm: 2991314.949142468
100000 completed. MAPE

### Задание на семинаре: попробовать улучшить метрику MAPE до 15.8% (топ-1 без ML с первой недели).

Варианты путей для улучшения:

    1) Делать новые признаки из существующих;
    2) Препроцессинг данных, целевой переменной - постпроцессинг ответов модели;
    3) Анализ ошибок модели –> генерация идей;
    4) Добавить регуляризацию;


<h3>Задание на семинаре: реализуем логистическую регрессию</h3>

<p>Мы получаем оптимальные веса алгоритма градиентным спуском:</p>

<p style="text-align:center"><br />
<br />
<span class="math-tex">\(\begin{bmatrix} w_{1}^{t+1}\\  ...\\ w_{m}^{t+1}\\  \end{bmatrix} = \begin{bmatrix} w_{1}^{t}\\  ...\\ w_{m}^{t}\\  \end{bmatrix} - \alpha \cdot  \begin{bmatrix} \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{1}^{(i)}\\  ...\\ \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})x_{m}^{(i)}\\  \end{bmatrix}\)</span></p>

<p style="text-align:center"><span class="math-tex">\(b^{t+1} = b^{t} - \alpha \sum_{i=1}^{n} (\frac{1}{1+exp(w^{T}x^{(i)})} - y^{(i)})\)</span></p>

<p style="text-align:center">&nbsp;</p>

<p>&nbsp;</p>

In [ ]:
from sklearn.metrics import accuracy_score


class LogisticRegression:
    
    def __init__(self, max_iter=1e4, lr=0.001, tol=0.001, print_every=100):
        
        '''
        max_iter – максимальное количеств
        '''
        
        self.max_iter = max_iter
        self.lr = lr
        self.tol = tol
        self.print_every = print_every
        
        self.weights = None
        self.bias = None
        
    def fit(self, X_train, y_train, X_val, y_val):
        
        '''
        Обучение модели.
        
        X_train – матрица объектов для обучения
        y_train – ответы на объектах для обучения
        
        X_val – матрица объектов для валидации
        y_val – ответы на объектах для валидации
        '''
        
        self.check_binary_clf_X_y(X_train, y_train)
        self.check_binary_clf_X_y(X_val, y_val)
                
        n, m = X_train.shape
        
        self.weights = 
        self.bias = 
        
        n_iter = 0
        gradient_norm = np.inf
        
        while n_iter < self.max_iter and gradient_norm > self.tol:
            
            dJdw, dJdb = self.grads(X_train, y_train)
            gradient_norm = np.linalg.norm(np.hstack([dJdw.flatten(), [dJdb]]))
                
            self.weights = 
            self.bias = 
            
            n_iter += 1
            
            if n_iter % self.print_every == 0:
                self.print_metrics(X_train, y_train, X_val, y_val, n_iter, gradient_norm)
        
        return self
    
    def predict(self, X):  
        
        '''
        Метод возвращает предсказанную метку класса на объектах X
        '''
        
        pass
        
    
    def predict_proba(self, X):
        
        '''
        Метод возвращает вероятность класса 1 на объектах X
        '''
        pass
    
    def grads(self, x, y):
        
        '''
        Рассчёт градиентов
        '''
        y_hat = 
        
        dJdw = 
        dJdb = 
        
        self.check_grads(dJdw, dJdb)
        
        return dJdw, dJdb
    
    @staticmethod
    def sigmoid(x):
        '''
        Сигмоида от x
        '''
        pass
    
    def print_metrics(self, X_train, y_train, X_val, y_val, n_iter, gradient_norm):
        
        train_preds = self.predict(X_train)
        val_preds = self.predict(X_val)
        
        train_acc = accuracy_score(train_preds, y_train)
        val_acc = accuracy_score(val_preds, y_val)
        
        print(f'{n_iter} completed. accuracy_score on train: {train_acc}, val: {val_acc}, grad_norm: {gradient_norm}')
        
    def check_grads(self, dJdw, dJdb):
        
        if not isinstance(dJdb, numbers.Real):
            raise ValueError(f'Производная по параметру b должна быть действительным'
                             f' числом, как и сам параметр b, а у нас {dJdb} типа {type(dJdb)}')
            
        if dJdw.shape != self.weights.shape:
            raise ValueError(f'Размерность градиента по параметрам w должна совпадать с самим вектором w, '
                             f'а у нас dJdw.shape = {dJdw.shape} не совпадает с weight.shape = {self.weights.shape}')
    
    @staticmethod
    def check_binary_clf_X_y(X, y):
        
        if X.shape[0] == 0:
            raise ValueError(f'X и y не должны быть пустыми, а у нас X.shape = {X.shape} и y.shape = {y.shape}')
            
        if np.isnan(X).any():
            raise ValueError(f'X не должен содержать "not a number" (np.nan)')
            
        if np.isnan(y).any():
            raise ValueError(f'y не должен содержать "not a number" (np.nan)')
        
        if X.shape[0] != y.shape[0]:
            raise ValueError(f'Длина X и y должна быть одинаковой, а у нас X.shape = {X.shape}, y.shape = {y.shape}')
            
        if y.shape[1] != 1:
            raise ValueError(f'y - вектор ответов должен быть размерности (m, 1), а у нас y.shape = {y.shape}')

                    
        if sorted(np.unique([1, 0, 0])) != [0, 1]:
            raise ValueError(f'Ответы на объектах должны быть только 0 или 1, а у нас np.unique(y) = {np.unique(y)}')


<h2>Домашнее задание</h2>

<p>Воспользуемся реализованной моделью логистической регрессии, чтобы решить задачу определения пола пользователя Авито.</p>

<p><a href="https://stepik.org/media/attachments/lesson/527992/binary_clf_data.csv" rel="noopener noreferrer nofollow">Данные</a> даны в сыром виде &ndash; айтемы и их категории, которые выкладывали покупатели на Авито. Целевая переменная: <em>gender.</em></p>

<p>Вам необходимо разбить данные на train, val. Перед загрузкой файла с ответом убедитесь, что точность (<a href="https://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html" rel="noopener noreferrer nofollow">accuracy</a>)&nbsp;на валидации не менее 0.7.</p>

<p>&nbsp;</p>

<p><strong>План действий</strong></p>

<p>Сначала нужно преобразовать категории с помощью one-hot encoding. Далее необходимо агрегировать категории, в которых пользователи выкладывали объявления, чтобы получить вектор признаков для каждого объекта. В результате у каждого пользователя будет вектор признаков, содержащий количество айтемов, выложенных в каждой из возможных категорий.</p>

<ul>
	<li>Убедитесь, что для каждого пользователя в выборке есть только один объект, каждый признак означает количество айтемов, выложенное этим пользователем в категории;</li>
	<li>Убедитесь, что после one-hot энкодинга каждая категория соответствует признаку,&nbsp;<strong>одинаковому в train, val и test.</strong></li>
</ul>

<p>Попробуйте варианты отбора признаков. Для борьбы с переобучением на редких категориях используйте регуляризацию. В качестве&nbsp;ответа загрузите файл с предсказанием пола для пользователей:</p>

<p style="text-align:center">&nbsp;</p>

<table align="center" border="1" cellpadding="1" cellspacing="1" style="width:500px">
	<thead>
		<tr>
			<th style="text-align:center">user_id</th>
			<th style="text-align:center">gender</th>
		</tr>
	</thead>
	<tbody>
		<tr>
			<td style="text-align:center">15424171</td>
			<td style="text-align:center">male</td>
		</tr>
		<tr>
			<td style="text-align:center">15454025</td>
			<td style="text-align:center">female</td>
		</tr>
	</tbody>
</table>

<p style="text-align:center">&nbsp;</p>

<p>Такой файл можно сформировать с помощью&nbsp;<code>test_predictions.to_csv(&#39;test_predictions.csv&#39;, index=False)</code>.</p>

<p>После того, как получилось обучить модель, ответьте на вопрос: какие из категорий вносят наибольший вклад в вероятность класса &quot;мужчина&quot; и класса &quot;женщина&quot;?</p>

<p>Например, если вы закодировали &quot;мужчина&quot; как 1, большие положительные веса при признаках будут означать большой вклад в вероятность класса 1, большие по модулю отрицательные веса будут вносить наибольший вклад в вероятность класса 0. Согласуется ли полученный результат с вашим жизненным опытом?</p>
